In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [2]:
seed = 13
tf.random.set_seed(seed)

In [3]:
data = pd.read_csv('..\\Data\\ze41_mol_desc_db_red.csv', header=0, sep=';', decimal=',')

In [4]:
col_names = data.columns
X = data[col_names[3:]].astype('float32')
y = data[col_names[2]].astype('float32')

In [5]:
X_scaled = MinMaxScaler(feature_range=(0,1)).fit_transform(X)

In [6]:
X_train, X_valid, y_train, y_valid = train_test_split(X_scaled, y, test_size=0.1, random_state=seed)
X_train = pd.DataFrame(X_train)
X_train.columns = X.columns

In [7]:
class Autoencoder(keras.models.Model):
    def __init__(self, latent_dim=5):
        super(Autoencoder, self).__init__()
        self.latent_dim = latent_dim   
        self.encoder = tf.keras.Sequential([
          keras.layers.Dense(latent_dim, activation='relu'),
        ])
        self.decoder = tf.keras.Sequential([
          keras.layers.Dense(len(X_train.columns), activation='linear')
        ])
    
    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [8]:
autoencoder = Autoencoder(latent_dim=5)

In [9]:
autoencoder.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.005),
    loss='mean_squared_error')

In [10]:
autoencoder.fit(X_train, X_train, validation_split=0.1, epochs=100)

Epoch 1/100
2/2 [==============================] - 0s 45ms/step - loss: 0.2253 - val_loss: 0.1974
Epoch 2/100
2/2 [==============================] - 0s 5ms/step - loss: 0.2087 - val_loss: 0.1751
Epoch 3/100
2/2 [==============================] - 0s 5ms/step - loss: 0.1843 - val_loss: 0.1479
Epoch 4/100
2/2 [==============================] - 0s 0s/step - loss: 0.1575 - val_loss: 0.1211
Epoch 5/100
2/2 [==============================] - 0s 0s/step - loss: 0.1311 - val_loss: 0.0966
Epoch 6/100
2/2 [==============================] - 0s 5ms/step - loss: 0.1081 - val_loss: 0.0770
Epoch 7/100
2/2 [==============================] - 0s 5ms/step - loss: 0.0901 - val_loss: 0.0624
Epoch 8/100
2/2 [==============================] - 0s 5ms/step - loss: 0.0769 - val_loss: 0.0522
Epoch 9/100
2/2 [==============================] - 0s 5ms/step - loss: 0.0678 - val_loss: 0.0455
Epoch 10/100
2/2 [==============================] - 0s 5ms/step - loss: 0.0619 - val_loss: 0.0411
Epoch 11/100
2/2 [============

Epoch 85/100
2/2 [==============================] - 0s 0s/step - loss: 0.0455 - val_loss: 0.0301
Epoch 86/100
2/2 [==============================] - 0s 0s/step - loss: 0.0455 - val_loss: 0.0300
Epoch 87/100
2/2 [==============================] - 0s 0s/step - loss: 0.0454 - val_loss: 0.0300
Epoch 88/100
2/2 [==============================] - 0s 5ms/step - loss: 0.0453 - val_loss: 0.0301
Epoch 89/100
2/2 [==============================] - 0s 5ms/step - loss: 0.0453 - val_loss: 0.0301
Epoch 90/100
2/2 [==============================] - 0s 5ms/step - loss: 0.0452 - val_loss: 0.0301
Epoch 91/100
2/2 [==============================] - 0s 5ms/step - loss: 0.0452 - val_loss: 0.0301
Epoch 92/100
2/2 [==============================] - 0s 5ms/step - loss: 0.0451 - val_loss: 0.0303
Epoch 93/100
2/2 [==============================] - 0s 5ms/step - loss: 0.0450 - val_loss: 0.0302
Epoch 94/100
2/2 [==============================] - 0s 5ms/step - loss: 0.0450 - val_loss: 0.0301
Epoch 95/100
2/2 [=====

In [11]:
best_cols = ['P_VSA_MR_5', 'Mor04m', 'E1p', 'CATS3D_02_AP', 'LUMO / eV']

In [12]:
X_sel = X_train[best_cols]

In [13]:
encoded = autoencoder.encoder(X_train.to_numpy()).numpy()

In [14]:
for i in range(5):
    print(sorted(X_sel.to_numpy()[i,:]), sorted(encoded[i, :]))

[0.0, 0.40519476, 0.43697143, 0.6193462, 0.66175306] [0.0, 0.0, 0.0, 0.0, 6.850691]
[0.3766234, 0.41165376, 0.5, 0.5158979, 0.56289303] [0.0, 0.0, 0.0, 0.0, 5.0634174]
[0.0, 0.023376644, 0.054961618, 0.5167059, 0.6462159] [0.0, 0.0, 0.0, 0.0, 5.8900356]
[0.0, 0.0, 0.18181819, 0.32409588, 0.567846] [0.0, 0.0, 0.0, 0.0, 5.499934]
[0.0, 0.0, 0.36883116, 0.48365432, 0.58608484] [0.0, 0.0, 0.0, 0.0, 1.4722421]


In [16]:
['P_VSA_MR_5'], 79/100
['P_VSA_MR_5', 'LUMO / eV'], 85/100
['P_VSA_MR_5', 'LUMO / eV', 'Mor04m'], 73/100
['P_VSA_MR_5', 'LUMO / eV', 'Mor04m', 'E1p'], 79/100
['P_VSA_MR_5', 'LUMO / eV', 'Mor04m', 'E1p', 'CATS3D_02_AP'], 40/100
['P_VSA_MR_5', 'LUMO / eV', 'Mor04m', 'E1p', 'CATS3D_02_AP', 'HOMO / eV'], 12/100

(['P_VSA_MR_5', 'LUMO / eV', 'Mor04m', 'E1p', 'CATS3D_02_AP', 'HOMO / eV'],
 0.12)